In [169]:
import pandas as pd

data = pd.read_csv("DataAll.csv")
keyTokens = pd.read_csv("NeighbourhoodCounts.csv")
antiTokens = pd.read_csv("AntiNeighbourCount.csv", header = None)
antiTokens = antiTokens.iloc[:,0].values

print (data.head())
print (keyTokens.head(10))
print (antiTokens.head())

for i in range(len(keyTokens["N=1"])):
    if (keyTokens["N=1"][0:i].sum()/float(keyTokens["N=1"].sum()) >=0.7):
        print ("---")
        print(i)
        break

   Unnamed: 0                                           TextOrig  \
0           0   Bed eight, Michael I Wu. Forty-eight years un...   
1           1   Vera Abbott, 93, bed four, under Dr Liu came ...   
2           2   Cindy Benedictos, bed 1, under Dr Smtih, came...   
3           3   This is bed 4. Jane morgan, 24, under Dr Tiln...   
4           4   Ramanathan Nisha Pillai, bed 1, 46 years old ...   

                                           TextToken                 Labels  
0  bed eight michael wu forty eight year under dr...                    NaN  
1  vera abbott bed four under dr liu come in with...                 nitros  
2  cindy benedictos bed under dr smtih come in fo...     education, insulin  
3  bed jane morgan under dr tilney with acute ren...  ivlabs, antibiotitics  
4  ramanathan nisha pillai bed year old under dr ...                    NaN  
   Unnamed: 0  N=1  N=2  N=3  N=4  N=5   N=6   N=7   N=8   N=9
0          mg  347  528  639  793  981  1095  1240  1386  14

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [83]:
# getting to see if the words are nouns, adverbs, adjectives
from nltk.corpus import wordnet as wn

# WordNet POS tags are: NOUN = 'n', ADJ = 's', VERB = 'v', ADV = 'r', ADJ_SAT = 'a'
# Descriptions (c) https://web.stanford.edu/~jurafsky/slp3/10.pdf
tag_map = {
        'JJ':wn.ADJ, # adjective (yellow)                         
        'NN':wn.NOUN, # noun, sing. or mass (llama)          
        'NNS':wn.NOUN, # noun, plural (llamas)                  
}

def get_wordnet_pos(tag):
    try:
        if tag_map[tag] != None:
            return tag_map[tag]
    except:
        return None
    return None

In [84]:
import nltk

def GetNoun (tokens):
    tokens = nltk.pos_tag(tokens)
    output = set()
    for i in tokens:
        if get_wordnet_pos(i[1]) == wn.NOUN:
            output.add(i[0])
    if len(output) >= 0:
        return list(output)

In [172]:
def GetMeds (neighbourhood = 1, numberOfWords = 1):
    meds = [[]]
    counter = 0
    for i in range(len(data)):
        tokenized = data["TextToken"][i].split(" ")
        nouns = []
        for keyToken in keyTokens.iloc[:73,0]:
            try:
                while keyToken in tokenized:
                    keyLocation = tokenized.index(keyToken)
                    tokenWindow = tokenized[max(keyLocation-neighbourhood,0):keyLocation+neighbourhood+1]
                    #tokenWindow = tokenized[keyLocation:keyLocation+neighbourhood+1]
                    tokenWindow.remove(keyToken)
                    for token in tokenWindow:
                        if token in antiTokens:
                            counter += 1
                            break
                    nouns += GetNoun(tokenWindow)
                    tokenized.pop(keyLocation)
            except ValueError:
                pass
        meds += [[",".join(list(set(nouns)))]]  
    print ("tokens removed:",counter)
    return meds
            

In [174]:
import time
startTime = time.time()
meds = GetMeds(3)
endTime = time.time()
print (endTime-startTime)

tokens removed: 20703
39.82011294364929


In [175]:
medsDF = pd.DataFrame(meds)
medsDF = medsDF.iloc[1:]
medsDF = medsDF.reset_index(drop = True)
medsDF.to_csv("tokenminer_results.csv", encoding = "utf-8")

In [38]:
data["TextToken"][1]

'vera abbott bed four under dr liu come in with chest pain with history of stroke previous chest pain asthma cataract glaucoma almost blind need assistance nitros with effect still under monitoring'

In [77]:
tags = nltk.pos_tag(['vera', 'abbott', 'bed', 'four', 'under'])
print (tags)
print (GetNoun(['vera', 'abbott', 'bed', 'four', 'under']))

[('vera', 'NN'), ('abbott', 'NN'), ('bed', 'VBD'), ('four', 'CD'), ('under', 'IN')]
{'vera'}
{'vera', 'abbott'}


TypeError: '>=' not supported between instances of 'set' and 'int'

In [150]:
if "of" in antiTokens.iloc[:,0].values:
    print ("yes")

yes
